## 전국평생학습강좌표준데이터

- drop: 교육시작시각, 교육종료시각, 강좌내용, 운영기관전화번호
- column 별 적절한 dtype 지정 --> astype() 이용
- (원본값 수정 방법, 1: 현재값 업데이트, 2: astype의 키워드 옵션 사용)
- (astype의 키워드 옵션 사용방법 --> Documentation 참조)
    * 수치값: np.int64 / np.float64
    * 종류를 구분하는 문자열(label): category
    * 날짜: np.datetime64
    * 이외: object
    
- 전체 DataFrame에 대한 describe() 결과 보기
- (describe 키워드, 옵션 --> Documentation 참조)
- 항목(column)별 결측치 / 오류치 / 이상치 개수 파악

In [156]:
import numpy as np
import pandas as pd

pd_data = pd.read_csv('전국평생학습강좌표준데이터.csv', encoding = 'cp949')

drop_list = ['교육시작시각', '교육종료시각', '강좌내용', '운영기관전화번호']
pd_data.drop(drop_list, axis = 1, inplace = True)       # inplace = 원본 수정

# print(pd_data.shape)
# print(pd_data.dtypes)
# print(pd_data.head(3))

# for e in enumerate(pd_data.columns):
#     print(e)

In [157]:
## dtypes 정리

for col_name in pd_data.columns:
    if '일자' in col_name:
        pd_data[col_name] = pd_data[col_name].astype(np.datetime64, copy = False)
    elif ('구분' in col_name) or ('여부' in col_name) or ('코드' in col_name):
        pd_data[col_name] = pd_data[col_name].astype('category', copy = False)
        
print(pd_data.dtypes)

강좌명                        object
강사명                        object
교육시작일자             datetime64[ns]
교육종료일자             datetime64[ns]
교육대상구분                   category
교육방법구분                   category
운영요일                       object
교육장소                       object
강좌정원수                       int64
수강료                         int64
교육장도로명주소                   object
운영기관명                      object
접수시작일자             datetime64[ns]
접수종료일자             datetime64[ns]
접수방법구분                   category
선정방법구분                   category
홈페이지주소                     object
직업능력개발훈련비지원강좌여부          category
학점은행제평가(학점)인정여부          category
평생학습계좌제평가인정여부            category
데이터기준일자            datetime64[ns]
제공기관코드                   category
제공기관명                      object
dtype: object


In [211]:
# pd_data.hasnans
t = []

for col_name in pd_data.columns:
    if pd_data[col_name].hasnans == True:
        print(col_name)
        cnt_s = pd_data[col_name].isna().value_counts()
        cnt = cnt_s.to_numpy()
        t.append([col_name, cnt[0], cnt[1]])
# print(t)
t = np.array(t)
missing_df = pd.DataFrame(t[:, 1:], index = t[:, 0], columns = ['value_count', 'NaN_count'])
# missing_df.index = missing_df['col_name']
print(missing_df)

운영요일
교육장소
접수시작일자
접수종료일자
선정방법구분
홈페이지주소
       value_count NaN_count
운영요일         22007      4528
교육장소         24706      1829
접수시작일자       21516      5019
접수종료일자       21474      5061
선정방법구분       22273      4262
홈페이지주소       24221      2314


In [183]:
# f1 = pd_data['운영요일'].isna()
# pd_data['운영요일'][f1]
pd_data['운영요일'].value_counts()

# 운영요일별 카운트시 운영요일을 특정할 수 없으므로,
# 삭제 조치하도록 함.

목          2556
화          2524
수          2434
월          1937
금          1851
           ... 
월+화+토         1
수요일+일요일       1
월요일+일요일       1
격주토요일         1
월 +수 +금       1
Name: 운영요일, Length: 218, dtype: int64

In [204]:
# pd_data['교육장소'].value_counts()

# '교육방법 구분'이 '온라인'으로 되어있을 경우, '온라인'으로 결측치를 대체
# 그렇지 않을 경우, 삭제.

pd_data['교육방법구분'].value_counts()
# pd_data['교육장소'].isna()
filter1 = pd_data['교육장소'].isna()
# print(pd_data['교육방법구분'][filter1].value_counts())

filter2 = pd_data['교육방법구분'] == '온라인'
pd_data['교육장소'][filter2] = '온라인'

pd_data['교육장소'].isna().value_counts()



# sub_data = pd_data[['교육방법구분', '교육장소']]
# x = sub_data.groupby('교육방법구분').sum()

# 접수시작/종료일자 결측치
# '교육방법 구분'이 '온라인'일 경우, 1월 1일부터 12월 31일까지의 값으로 대체.
# 그렇지 않을 경우, 삭제.

C:\Users\ITPS\AppData\Local\Temp\ipykernel_4888\2089335317.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['교육장소'][filter2] = '온라인'


False    24706
True      1829
Name: 교육장소, dtype: int64

In [210]:
for i in range(pd_data.shape[0]):
    row = pd_data.iloc[i, :]
    if (row['교육장소'] == np.NaN) and (row['교육방법구분'] == '온라인'):
        row['교육장소'] = '온라인'

In [185]:
pd_data['선정방법구분'].value_counts()
# 결측치, 삭제

선착순         16861
추첨           1219
제한없음          516
추첨+선착순        451
인터넷 추첨        392
            ...  
방문접수후 면접        1
방문심사            1
읍면 추천           1
군부대별 선정         1
자체심사            1
Name: 선정방법구분, Length: 72, dtype: int64

In [125]:
## 4번, 5번, 6번, 15번 제외

# pd_data.dtypes
lista = list(pd_data.columns)

# for e in pd_data.columns[4:7]:
#     lista.remove(e)
    
# lista.remove(pd_data.columns[15])
print(lista)

print(len(pd_data.columns), len(lista))

pd_data = pd_data[lista]
print(pd_data.dtypes)

['강좌명', '강사명', '교육시작일자', '교육종료일자', '교육대상구분', '교육방법구분', '운영요일', '교육장소', '강좌정원수', '수강료', '교육장도로명주소', '운영기관명', '접수시작일자', '접수종료일자', '접수방법구분', '선정방법구분', '홈페이지주소', '직업능력개발훈련비지원강좌여부', '학점은행제평가(학점)인정여부', '평생학습계좌제평가인정여부', '데이터기준일자', '제공기관코드', '제공기관명']
23 23
강좌명                object
강사명                object
교육시작일자             object
교육종료일자             object
교육대상구분             object
교육방법구분             object
운영요일               object
교육장소               object
강좌정원수               int64
수강료                 int64
교육장도로명주소           object
운영기관명              object
접수시작일자             object
접수종료일자             object
접수방법구분             object
선정방법구분             object
홈페이지주소             object
직업능력개발훈련비지원강좌여부    object
학점은행제평가(학점)인정여부    object
평생학습계좌제평가인정여부      object
데이터기준일자            object
제공기관코드             object
제공기관명              object
dtype: object


In [128]:
## 2, 3, 12, 13, 20 datetime. 21 int
print(pd_data)
pd_data.dtypes



                                    강좌명      강사명      교육시작일자      교육종료일자  \
0      오늘의 점심레시피 - 곰돌이 함박스테이크, 고슴도치 주먹밥  MBC씨앤아이  2022-01-01  2022-12-31   
1            오늘의 점심레시피 - 과일 볶음밥 및 감자 피자  MBC씨앤아이  2022-01-01  2022-12-31   
2                    오늘의 점심레시피 - 과일채소 쌈  MBC씨앤아이  2022-01-01  2022-12-31   
3                  오늘의 점심레시피 - 꼬치 궁중떡볶이  MBC씨앤아이  2022-01-01  2022-12-31   
4        오늘의 점심레시피 - 냉 녹차밥, 매콤한 해산물 샐러드  MBC씨앤아이  2022-01-01  2022-12-31   
...                                 ...      ...         ...         ...   
26530                            도자기페인팅      도선화  2022-11-17  2022-12-15   
26531                            나이프페인팅      조혜수  2022-10-07  2022-11-25   
26532                        홈스타일링 마크라메      권철은  2022-10-07  2022-12-09   
26533                   [자격증]원예치료전문가 3급      김세현  2022-10-07  2022-12-09   
26534                [자격증]그림책감정코칭지도사 2급      김은경  2022-10-07  2022-11-25   

      교육대상구분 교육방법구분           운영요일        교육장소  강좌정원수    수강료  ...      접수종료일자  \
0     

강좌명                object
강사명                object
교육시작일자             object
교육종료일자             object
교육대상구분             object
교육방법구분             object
운영요일               object
교육장소               object
강좌정원수               int64
수강료                 int64
교육장도로명주소           object
운영기관명              object
접수시작일자             object
접수종료일자             object
접수방법구분             object
선정방법구분             object
홈페이지주소             object
직업능력개발훈련비지원강좌여부    object
학점은행제평가(학점)인정여부    object
평생학습계좌제평가인정여부      object
데이터기준일자            object
제공기관코드             object
제공기관명              object
dtype: object

In [126]:
for i in enumerate(pd_data.columns):
    print(i)

(0, '강좌명')
(1, '강사명')
(2, '교육시작일자')
(3, '교육종료일자')
(4, '교육대상구분')
(5, '교육방법구분')
(6, '운영요일')
(7, '교육장소')
(8, '강좌정원수')
(9, '수강료')
(10, '교육장도로명주소')
(11, '운영기관명')
(12, '접수시작일자')
(13, '접수종료일자')
(14, '접수방법구분')
(15, '선정방법구분')
(16, '홈페이지주소')
(17, '직업능력개발훈련비지원강좌여부')
(18, '학점은행제평가(학점)인정여부')
(19, '평생학습계좌제평가인정여부')
(20, '데이터기준일자')
(21, '제공기관코드')
(22, '제공기관명')
